1.1 Data Load

sms_spam.scv 데이터는 문자 내용이 스팸인지 아닌지를 구분하기 위한 데이터임

In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2021)

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [65]:
spam = pd.read_csv("sms_spam.csv")

In [66]:
text = spam["text"]
label = spam["type"]

1.2 Data EDA

In [67]:
text[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [68]:
label[0]
# ham = 스팸이 아니다
# spam = 스팸 이다

'ham'

In [69]:
label.value_counts()

ham     4827
spam     747
Name: type, dtype: int64

1.3 Data Cleansing

정답의 문자를 숫자로 변환시켜줍니다.\
ham은 0으로, spam은 1로 변환 시켜주겠습니다.

In [70]:
# map을 활용해서 dictionary 으로 변환
label = label.map({"ham":0,"spam":1})

In [71]:
label.value_counts()

0    4827
1     747
Name: type, dtype: int64

text를 문자만 존재하도록 정리하도록 해줍니다.\
regex를 통해 영어, 숫자 그리고 띄어쓰기를 제외한 모든 단어를 지우도록 하겠습니다.

In [72]:
re_pattern = "[^a-zA-Z0-9\ ]"

In [73]:
text[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [74]:
text.iloc[:1].str.replace(re_pattern, "", regex=True)[0]

'Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat'

In [75]:
text = text.str.replace(re_pattern, "", regex=True)

그리고 나서 대문자들을 모두 소문자로 바꿔 줍니다.

In [76]:
text[0]

'Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat'

In [77]:
text.iloc[:1].str.lower()[0]

'go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat'

In [78]:
text = text.str.lower()

In [79]:
text[0]

'go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat'

1.4 Data Split

In [80]:
from sklearn.model_selection import train_test_split

train_text, test_text, train_label, test_label = train_test_split(
    text, label, train_size=0.7, random_state=2021
)

In [81]:
print(f"train_data size: {len(train_label)}, {len(train_label)/len(text):.2f}")
print(f"test_data size: {len(test_label)}, {len(test_label)/len(text):.2f}")

train_data size: 3901, 0.70
test_data size: 1673, 0.30


# 2. Count Vectorize

이제 Naive Bayes를 학습시키기 위해서 각 문장에서 단어들이 몇 번 나왔는지로 변환해야 함

2.1 word tokenize\
문장을 단어로 나누는 데에는 nltk 패키지의 word_tokenize를 이용함

In [82]:
import nltk
from nltk import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [83]:
# 자연어 데이터는 바로 나이브 베이즈 모델에 넣을 수 없음!!
# 모델에 넣기 전에 Count Vectorize 메트릭스 형태로 들어가야함

train_text.iloc[0]

'am only searching for good dual sim mobile pa'

In [84]:
# 띄어쓰기 단위로 데이터 나뉨
word_tokenize(train_text.iloc[0])

['am', 'only', 'searching', 'for', 'good', 'dual', 'sim', 'mobile', 'pa']

2.2 count vectorize\
다음은 sklearn.feature_extractation.text의 CountVectorizer를 이용해 단어들을 count vector로 만들어 보겠음

In [85]:
from sklearn.feature_extraction.text import CountVectorizer

우선 예시로 2개의 문장으로 CountVectorizer를 학습해 보겠습니다.

In [86]:
train_text.iloc[:2].values

array(['am only searching for good dual sim mobile pa',
       'excellent ill see what rileys plans are'], dtype=object)

In [87]:
# countvectorizer에 tokenizer 을 word tokenize로 하겟다.
cnt_vectorizer = CountVectorizer(tokenizer=word_tokenize)

In [88]:
cnt_vectorizer.fit(train_text.iloc[:2])

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CountVectorizer(tokenizer=<function word_tokenize at 0x7fd5fdabe3a0>)

문장에서 나온 단어들은 다음과 같음

In [89]:
cnt_vectorizer.vocabulary_
# 2문장에서 나온 단어들을ㄴ 총 15개, 앞에 단어가 뒤에 번호로 매핑됨

{'am': 0,
 'only': 8,
 'searching': 12,
 'for': 4,
 'good': 5,
 'dual': 2,
 'sim': 14,
 'mobile': 7,
 'pa': 9,
 'excellent': 3,
 'ill': 6,
 'see': 13,
 'what': 15,
 'rileys': 11,
 'plans': 10,
 'are': 1}

In [90]:
vocab = sorted(cnt_vectorizer.vocabulary_.items(), key=lambda x: x[1])
vocab = list(map(lambda x: x[0], vocab))
vocab

['am',
 'are',
 'dual',
 'excellent',
 'for',
 'good',
 'ill',
 'mobile',
 'only',
 'pa',
 'plans',
 'rileys',
 'searching',
 'see',
 'sim',
 'what']

In [91]:
# fitting한 count vectorizer로 변환
sample_cnt_vector = cnt_vectorizer.transform(train_text.iloc[:2]).toarray()
sample_cnt_vector

array([[1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1]])

In [92]:
train_text.iloc[:2].values

array(['am only searching for good dual sim mobile pa',
       'excellent ill see what rileys plans are'], dtype=object)

2.2.1 학습

이제 모든 데이터에 대해서 진행

In [93]:
cnt_vectorizer = CountVectorizer(tokenizer=word_tokenize)
cnt_vectorizer.fit(train_text)

CountVectorizer(tokenizer=<function word_tokenize at 0x7fd5fdabe3a0>)

In [94]:
# 전체 단어는 7908개가 존재
len(cnt_vectorizer.vocabulary_)

7908

2.2.2 예측

In [101]:
train_matrix = cnt_vectorizer.transform(train_text)
test_matrix = cnt_vectorizer.transform(test_text)

만약 존재하지 않는 단어가 들어올 경우 어떻게 될까?\
CountVectorize는 학습한 단어장에 존재하지 않는 단어가 들어오게 될 경우 무시함

In [102]:
cnt_vectorizer.transform(["notavilblewordforcnt"]).toarray().sum() #??

0

3. Naive Bayes

분류를 위한 Naive Bayes모델은 sklearn.naive_bayes의 BernoulliNB 사용하면 됨


In [103]:
from sklearn.naive_bayes import BernoulliNB

naive_bayes = BernoulliNB()

3.1 학습

In [104]:
naive_bayes.fit(train_matrix, train_label)

BernoulliNB()

3.2 예측

In [105]:
train_pred = naive_bayes.predict(train_matrix)
test_pred = naive_bayes.predict(test_matrix)
                          

3.3 평가

In [107]:
from sklearn.metrics import accuracy_score

train_acc = accuracy_score(train_label, train_pred)
test_acc = accuracy_score(test_label, test_pred)

In [109]:
print(f"Train Accuracy is {train_acc:.4f}")
print(f"Test Accuracy is {test_acc:.4f}")

Train Accuracy is 0.9854
Test Accuracy is 0.9767
